In [ ]:
#1.2
import math
from bloqade import move
from kirin.passes import aggressive
from iquhack_scoring import MoveScorer
pi = math.pi
#Have to move to gate zone to execute
#Leaves qbit in gate zone afterwards
@move.vmove
def local_hadamard(state: move.core.AtomState, indices) -> move.core.AtomState:
    state = move.LocalXY(state, 0.25 * pi, 0.5 * pi, indices)
    state = move.LocalRz(state, pi, indices)
    state = move.LocalXY(state, -0.25 * pi, 0.5 * pi, indices)
    return state

#Make sure gate zone has no pairs, else those pairs also will get CZ'ed
#Takes qubits back afterwards
@move.vmove
def cx_layer(state: move.core.AtomState, storage_site: int, gate_index: int):
    state.gate[[gate_index]] = move.Move(state.storage[[storage_site]])
    state = local_hadamard(state, [gate_index])
    state = move.GlobalCZ(state)
    state = local_hadamard(state, [gate_index])
    state.storage[[storage_site]] = move.Move(state.gate[[gate_index]])
    return 
 
@move.vmove
def main():
    q = move.NewQubitRegister(3)
    state = move.Init(qubits=[q[0],q[1],q[2]], indices=[0,1,2])
    #state.gate[[]] = move.Move(state.storage[[]])
    #state = move.LocalXY(atom_state=state,x_exponent=1,axis_phase_exponent=,indices=[0,3])
    #state = move.GlobalCZ(state)
    state.gate[[0]] = move.Move(state.storage[[2]])
    state = local_hadamard(state,[0])
    
    state = cx_layer(state,1,1)

    state = move.LocalRz(state,-0.25*pi,indecis=[0])
    
    state = cx_layer(state,0,1)

    state = move.LocalRz(state,0.25*pi,indecis=[0])

    state = cx_layer(state,1,1)
    
    state = move.LocalRz(state,-0.25*pi,indecis=[0])

    state.gate[[1]] = move.Move(state.storage[[1]])
    state = move.LocalRz(state,0.25*pi,indecis=[1])
    state.storage[[1]] = move.Move(state.gate[[1]])

    state = cx_layer(state, 0,1)
    
    state = move.LocalRz(state,0.25*pi,indecis=[0]) 
    state.storage[[0]] = move.Move(state.gate[[2]])

    state.gate[[0]] = move.Move(state.storage[[0]])
    state = cx_layer(state,1,1)

    state = move.LocalRz(state,0.25*pi,indecis=[0])
    state = move.LocalRz(state,0.25*pi,indecis=[1])

    state.storage[[0,1]] = move.Move(state.gate[[0,1]])

    state = local_hadamard(state,3)
    
    move.Execute(state)

print(MoveScorer(main,expected_qasm=expected_qasm).score())